In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.expand_frame_repr',False)
pd.set_option('max_colwidth',1)

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [3]:
df = pd.read_csv(r"C:\Users\ANAIKUTTY\Documents\Cricket analysis\data files\IPL_ball_by_ball_updated.csv")

In [4]:
df.head(1)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,335982,2008,2008-04-18,M Chinnaswamy Stadium,2,6.8,Royal Challengers Bangalore,Kolkata Knight Riders,MV Boucher,CL White,AB Agarkar,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df['batting_team'] = df['batting_team'].replace({'Delhi Daredevils':'Delhi Capitals','Kings XI Punjab':'Punjab Kings','Rising Pune Supergiant':'Rising Pune Supergiants'})
df['bowling_team'] = df['bowling_team'].replace({'Delhi Daredevils':'Delhi Capitals','Kings XI Punjab':'Punjab Kings','Rising Pune Supergiant':'Rising Pune Supergiants'})

In [6]:
df['batting_team'].unique()

array(['Royal Challengers Bangalore', 'Kolkata Knight Riders',
       'Rajasthan Royals', 'Delhi Capitals', 'Punjab Kings',
       'Chennai Super Kings', 'Deccan Chargers', 'Mumbai Indians',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Rising Pune Supergiants', 'Gujarat Lions', 'Gujarat Titans',
       'Lucknow Super Giants'], dtype=object)

In [7]:
df['tot_runs'] = df['runs_off_bat'] + df['extras']

In [8]:
df['is_out'] = np.where(df['player_dismissed'].fillna(0) == 0, 0, 1)

In [9]:
df['is_ball'] = np.where((df['wides'].fillna(0)==0)&(df['noballs'].fillna(0)==0), 1,0)

In [10]:
df = df[['match_id','season','innings', 'batting_team', 'bowling_team','tot_runs','is_out', 'is_ball']]

In [11]:
df.head(1)

,match_id,season,innings,batting_team,bowling_team,tot_runs,is_out,is_ball
0,335982,2008,2,Royal Challengers Bangalore,Kolkata Knight Riders,4,0,1


In [12]:
df['total'] = df.groupby(['match_id','innings'])['tot_runs'].transform('sum')

In [13]:
df['total_score'] = df.groupby(['match_id','innings'])['tot_runs'].cumsum()

In [14]:
df['wicket'] = df.groupby(['match_id','innings'])['is_out'].transform('sum')

In [15]:
df.head()

,match_id,season,innings,batting_team,bowling_team,tot_runs,is_out,is_ball,total,total_score,wicket
0,335982,2008,2,Royal Challengers Bangalore,Kolkata Knight Riders,4,0,1,82,4,10
1,335982,2008,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,0,1,82,5,10
2,335982,2008,2,Royal Challengers Bangalore,Kolkata Knight Riders,2,0,1,82,7,10
3,335982,2008,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,0,1,82,8,10
4,335982,2008,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,0,1,82,8,10


In [16]:
df['tot_wkt'] = df.groupby(['match_id','innings'])['is_out'].cumsum()

In [17]:
df['over'] = (df.groupby(['match_id','innings'])['is_ball'].cumsum() / 6).round(2)

In [18]:
df['is_bound'] = np.where(df['tot_runs'].isin([4,6]),1,0)

In [19]:
df['is_dot'] = np.where(df['tot_runs'].isin([0]),1,0)

In [20]:
df['last_30_dot'] = df.groupby(['match_id','innings'])['is_dot'].transform(lambda x: x.rolling(min_periods = 1,window=30).sum())

In [21]:
#df['tot_bound'] = df.groupby(['match_id','innings'])['is_bound'].cumsum()

In [22]:
df['last_30_runs'] = df.groupby(['match_id','innings'])['tot_runs'].transform(lambda x: x.rolling(min_periods = 1,window=30).sum())

In [23]:
df['last_30_wkt'] = df.groupby(['match_id','innings'])['is_out'].transform(lambda x: x.rolling(min_periods = 1,window=30).sum())

In [24]:
df['last_30_bound'] =df.groupby(['match_id','innings'])['is_bound'].transform(lambda x: x.rolling(min_periods = 1,window=30).sum())

In [25]:
df['tgt'] = np.where(df['innings']==1,0,df.groupby(['match_id','innings'])['tot_runs'].transform('sum'))

In [26]:
df['tgt'] = np.where(df['innings']==1,0,df['tgt']+1)

In [27]:
df.tail(5)

,match_id,season,innings,batting_team,bowling_team,tot_runs,is_out,is_ball,total,total_score,wicket,tot_wkt,over,is_bound,is_dot,last_30_dot,last_30_runs,last_30_wkt,last_30_bound,tgt
225948,1312200,2022,1,Rajasthan Royals,Gujarat Titans,1,0,1,130,128,9,9,19.50,0,0,15.0,25.0,3.0,3.0,0
225949,1312200,2022,1,Rajasthan Royals,Gujarat Titans,1,0,1,130,129,9,9,19.67,0,0,14.0,26.0,3.0,3.0,0
225950,1312200,2022,1,Rajasthan Royals,Gujarat Titans,1,0,1,130,130,9,9,19.83,0,0,13.0,27.0,3.0,3.0,0
225951,1312200,2022,1,Rajasthan Royals,Gujarat Titans,0,0,1,130,130,9,9,20.00,0,1,13.0,27.0,2.0,3.0,0
225952,1312200,2022,2,Gujarat Titans,Rajasthan Royals,6,0,1,133,133,3,3,18.17,1,0,6.0,47.0,0.0,4.0,134


In [28]:
df = df.drop(columns=['season'])

In [29]:
#df = pd.get_dummies(data=df, columns=['batting_team','bowling_team'])

In [30]:
df.head(1)

,match_id,innings,batting_team,bowling_team,tot_runs,is_out,is_ball,total,total_score,wicket,tot_wkt,over,is_bound,is_dot,last_30_dot,last_30_runs,last_30_wkt,last_30_bound,tgt
0,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,4,0,1,82,4,10,0,0.17,1,0,0.0,4.0,0.0,1.0,83


In [31]:
#len(df.columns)

In [32]:
'''
df = df[['batting_team_Chennai Super Kings', 'batting_team_Deccan Chargers',
       'batting_team_Delhi Capitals', 'batting_team_Gujarat Lions',
       'batting_team_Gujarat Titans', 'batting_team_Kochi Tuskers Kerala',
       'batting_team_Kolkata Knight Riders',
       'batting_team_Lucknow Super Giants', 'batting_team_Mumbai Indians',
       'batting_team_Pune Warriors', 'batting_team_Punjab Kings',
       'batting_team_Rajasthan Royals', 'batting_team_Rising Pune Supergiants',
       'batting_team_Royal Challengers Bangalore',
       'batting_team_Sunrisers Hyderabad', 'bowling_team_Chennai Super Kings',
       'bowling_team_Deccan Chargers', 'bowling_team_Delhi Capitals', 'bowling_team_Gujarat Lions',
       'bowling_team_Gujarat Titans',
       'bowling_team_Kochi Tuskers Kerala',
       'bowling_team_Kolkata Knight Riders',
       'bowling_team_Lucknow Super Giants', 'bowling_team_Mumbai Indians',
       'bowling_team_Pune Warriors', 'bowling_team_Punjab Kings',
       'bowling_team_Rajasthan Royals', 
       'bowling_team_Rising Pune Supergiants',
       'bowling_team_Royal Challengers Bangalore',
       'bowling_team_Sunrisers Hyderabad','over','total_score','tot_wkt','last_30_runs','last_30_wkt','last_30_dot','last_30_bound','total']]
       '''


"\ndf = df[['batting_team_Chennai Super Kings', 'batting_team_Deccan Chargers',\n       'batting_team_Delhi Capitals', 'batting_team_Gujarat Lions',\n       'batting_team_Gujarat Titans', 'batting_team_Kochi Tuskers Kerala',\n       'batting_team_Kolkata Knight Riders',\n       'batting_team_Lucknow Super Giants', 'batting_team_Mumbai Indians',\n       'batting_team_Pune Warriors', 'batting_team_Punjab Kings',\n       'batting_team_Rajasthan Royals', 'batting_team_Rising Pune Supergiants',\n       'batting_team_Royal Challengers Bangalore',\n       'batting_team_Sunrisers Hyderabad', 'bowling_team_Chennai Super Kings',\n       'bowling_team_Deccan Chargers', 'bowling_team_Delhi Capitals', 'bowling_team_Gujarat Lions',\n       'bowling_team_Gujarat Titans',\n       'bowling_team_Kochi Tuskers Kerala',\n       'bowling_team_Kolkata Knight Riders',\n       'bowling_team_Lucknow Super Giants', 'bowling_team_Mumbai Indians',\n       'bowling_team_Pune Warriors', 'bowling_team_Punjab Kings',

In [33]:
#len(df.columns)

In [34]:
'''
x = df.drop(columns=['total'])
y = df['total'].values
'''

"\nx = df.drop(columns=['total'])\ny = df['total'].values\n"

In [35]:
#len(x.columns)

In [36]:
#for i in range(len(x.columns)):
   # print(i,[x.columns][i])

In [37]:
'''
X_train,X_test, y_train,y_test = train_test_split(x,y,test_size=0.25)
'''

'\nX_train,X_test, y_train,y_test = train_test_split(x,y,test_size=0.25)\n'

In [38]:
#model = LinearRegression()

In [39]:
#model.fit(X_train,y_train)

In [40]:
#y_pred = model.predict(X_test)
#mean_absolute_error(y_test,y_pred)

In [41]:
#df['batting_team'].unique()

In [42]:
def score_prediction(df,Bat_team, Bowl_team,innings, overs,total_score,tot_wkt,last_30_runs,last_30_dot,last_30_wkt,last_30_bound,tgt):

    df = pd.get_dummies(data=df, columns=['batting_team','bowling_team'])

    df = df.drop(columns=['is_out','is_ball','is_bound','is_dot','match_id'])

    df = df[['batting_team_Chennai Super Kings', 'batting_team_Deccan Chargers',
        'batting_team_Delhi Capitals', 'batting_team_Gujarat Lions',
        'batting_team_Gujarat Titans', 'batting_team_Kochi Tuskers Kerala',
        'batting_team_Kolkata Knight Riders',
        'batting_team_Lucknow Super Giants', 'batting_team_Mumbai Indians',
        'batting_team_Pune Warriors', 'batting_team_Punjab Kings',
        'batting_team_Rajasthan Royals', 'batting_team_Rising Pune Supergiants',
        'batting_team_Royal Challengers Bangalore',
        'batting_team_Sunrisers Hyderabad', 'bowling_team_Chennai Super Kings',
        'bowling_team_Deccan Chargers', 'bowling_team_Delhi Capitals', 'bowling_team_Gujarat Lions',
        'bowling_team_Gujarat Titans',
        'bowling_team_Kochi Tuskers Kerala',
        'bowling_team_Kolkata Knight Riders',
        'bowling_team_Lucknow Super Giants', 'bowling_team_Mumbai Indians',
        'bowling_team_Pune Warriors', 'bowling_team_Punjab Kings',
        'bowling_team_Rajasthan Royals', 
        'bowling_team_Rising Pune Supergiants',
        'bowling_team_Royal Challengers Bangalore',
        'bowling_team_Sunrisers Hyderabad','innings','over','total_score','tot_wkt','last_30_runs','last_30_wkt','last_30_dot','last_30_bound','total','tgt']]

    x = df.drop(columns=['total'])
    y = df['total'].values



    X_train,X_test, y_train,y_test = train_test_split(x,y,test_size=0.25)
    model = LinearRegression()
    model.fit(X_train,y_train)

    y_pred = model.predict(X_test)
    mean_absolute_error(y_test,y_pred)

    temp = []

    if Bat_team == 'Chennai Super Kings':
        temp += [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif Bat_team == 'Deccan Chargers':
        temp += [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif Bat_team == 'Delhi Capitals':
        temp += [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0]
    elif Bat_team == 'Gujarat Lions':
        temp += [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0]
    elif Bat_team == 'Gujarat Titans':
        temp += [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]
    elif Bat_team == 'Kochi Tuskers Kerala':
        temp += [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0]
    elif Bat_team == 'Kolkata Knight Riders':
        temp += [0,0,0,0,0,0,1,0,0,0,0,0,0,0,0]
    elif Bat_team == 'Lucknow Super Giants':
        temp += [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]
    elif Bat_team == 'Mumbai Indians':
        temp += [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0] 
    elif Bat_team == 'Pune Warriors':
        temp += [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]
    elif Bat_team == 'Punjab Kings':
        temp += [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]
    elif Bat_team == 'Rajasthan Royals':
        temp += [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0]
    elif Bat_team == 'Rising Pune Supergiants':
        temp += [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0]
    elif Bat_team == 'Royal Challengers Bangalore':
        temp += [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]
    elif Bat_team == 'Sunrisers Hyderabad':
        temp += [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]

    
    if Bowl_team == 'Chennai Super Kings':
        temp += [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif Bowl_team == 'Deccan Chargers':
        temp += [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif Bowl_team == 'Delhi Capitals':
        temp += [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0]
    elif Bowl_team == 'Gujarat Lions':
        temp += [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0]
    elif Bowl_team == 'Gujarat Titans':
        temp += [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]
    elif Bowl_team == 'Kochi Tuskers Kerala':
        temp += [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0]
    elif Bowl_team == 'Kolkata Knight Riders':
        temp += [0,0,0,0,0,0,1,0,0,0,0,0,0,0,0]
    elif Bowl_team == 'Lucknow Super Giants':
        temp += [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]
    elif Bowl_team == 'Mumbai Indians':
        temp += [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0] 
    elif Bowl_team == 'Pune Warriors':
        temp += [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]
    elif Bowl_team == 'Punjab Kings':
        temp += [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]
    elif Bowl_team == 'Rajasthan Royals':
        temp += [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0]
    elif Bowl_team == 'Rising Pune Supergiants':
        temp += [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0]
    elif Bowl_team == 'Royal Challengers Bangalore':
        temp += [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]
    elif Bowl_team == 'Sunrisers Hyderabad':
        temp += [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]

    temp += [innings,overs , total_score , tot_wkt, last_30_runs, last_30_wkt, last_30_dot,last_30_bound,tgt]

    print(int(model.predict([temp])))
    print('Score Range',int(model.predict([temp]))-18,' to ',int(model.predict([temp]))+18)



In [43]:
Bat_team = 'Sunrisers Hyderabad'
Bowl_team= 'Gujarat Titans'
innings = 1
overs = 5
total_score = 38
tot_wkt = 2
last_30_runs = 38
last_30_dot = 15
last_30_wkt = 2
last_30_bound = 8
tgt = 120

#Machine Learning With additional two features Innings,Target

score_prediction(df,Bat_team, Bowl_team, innings,overs, total_score, tot_wkt, last_30_runs, last_30_dot, last_30_wkt, last_30_bound,tgt)

230
Score Range 212  to  248


In [44]:
def wkt_prediction(df,Bat_team, Bowl_team,innings, overs,total_score,tot_wkt,last_30_runs,last_30_dot,last_30_wkt,last_30_bound,tgt):


    df = pd.get_dummies(data=df, columns=['batting_team','bowling_team'])

    df = df.drop(columns=['is_out','is_ball','is_bound','is_dot','match_id'])

    df = df[['batting_team_Chennai Super Kings', 'batting_team_Deccan Chargers',
        'batting_team_Delhi Capitals', 'batting_team_Gujarat Lions',
        'batting_team_Gujarat Titans', 'batting_team_Kochi Tuskers Kerala',
        'batting_team_Kolkata Knight Riders',
        'batting_team_Lucknow Super Giants', 'batting_team_Mumbai Indians',
        'batting_team_Pune Warriors', 'batting_team_Punjab Kings',
        'batting_team_Rajasthan Royals', 'batting_team_Rising Pune Supergiants',
        'batting_team_Royal Challengers Bangalore',
        'batting_team_Sunrisers Hyderabad', 'bowling_team_Chennai Super Kings',
        'bowling_team_Deccan Chargers', 'bowling_team_Delhi Capitals', 'bowling_team_Gujarat Lions',
        'bowling_team_Gujarat Titans',
        'bowling_team_Kochi Tuskers Kerala',
        'bowling_team_Kolkata Knight Riders',
        'bowling_team_Lucknow Super Giants', 'bowling_team_Mumbai Indians',
        'bowling_team_Pune Warriors', 'bowling_team_Punjab Kings',
        'bowling_team_Rajasthan Royals', 
        'bowling_team_Rising Pune Supergiants',
        'bowling_team_Royal Challengers Bangalore',
        'bowling_team_Sunrisers Hyderabad','innings','over','total_score','tot_wkt','last_30_runs','last_30_wkt','last_30_dot','last_30_bound','wicket','tgt']]

    x = df.drop(columns=['wicket'])
    y = df['wicket'].values



    X_train,X_test, y_train,y_test = train_test_split(x,y,test_size=0.25)
    model = LinearRegression()
    model.fit(X_train,y_train)

    y_pred = model.predict(X_test)
    mean_absolute_error(y_test,y_pred)
        
    temp = []

    if Bat_team == 'Chennai Super Kings':
        temp += [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif Bat_team == 'Deccan Chargers':
        temp += [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif Bat_team == 'Delhi Capitals':
        temp += [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0]
    elif Bat_team == 'Gujarat Lions':
        temp += [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0]
    elif Bat_team == 'Gujarat Titans':
        temp += [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]
    elif Bat_team == 'Kochi Tuskers Kerala':
        temp += [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0]
    elif Bat_team == 'Kolkata Knight Riders':
        temp += [0,0,0,0,0,0,1,0,0,0,0,0,0,0,0]
    elif Bat_team == 'Lucknow Super Giants':
        temp += [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]
    elif Bat_team == 'Mumbai Indians':
        temp += [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0] 
    elif Bat_team == 'Pune Warriors':
        temp += [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]
    elif Bat_team == 'Punjab Kings':
        temp += [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]
    elif Bat_team == 'Rajasthan Royals':
        temp += [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0]
    elif Bat_team == 'Rising Pune Supergiants':
        temp += [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0]
    elif Bat_team == 'Royal Challengers Bangalore':
        temp += [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]
    elif Bat_team == 'Sunrisers Hyderabad':
        temp += [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]

    
    if Bowl_team == 'Chennai Super Kings':
        temp += [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif Bowl_team == 'Deccan Chargers':
        temp += [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif Bowl_team == 'Delhi Capitals':
        temp += [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0]
    elif Bowl_team == 'Gujarat Lions':
        temp += [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0]
    elif Bowl_team == 'Gujarat Titans':
        temp += [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]
    elif Bowl_team == 'Kochi Tuskers Kerala':
        temp += [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0]
    elif Bowl_team == 'Kolkata Knight Riders':
        temp += [0,0,0,0,0,0,1,0,0,0,0,0,0,0,0]
    elif Bowl_team == 'Lucknow Super Giants':
        temp += [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]
    elif Bowl_team == 'Mumbai Indians':
        temp += [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0] 
    elif Bowl_team == 'Pune Warriors':
        temp += [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]
    elif Bowl_team == 'Punjab Kings':
        temp += [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]
    elif Bowl_team == 'Rajasthan Royals':
        temp += [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0]
    elif Bowl_team == 'Rising Pune Supergiants':
        temp += [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0]
    elif Bowl_team == 'Royal Challengers Bangalore':
        temp += [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]
    elif Bowl_team == 'Sunrisers Hyderabad':
        temp += [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]

    temp += [innings,overs , total_score , tot_wkt, last_30_runs, last_30_wkt, last_30_dot,last_30_bound,tgt]

    print(int(model.predict([temp])))
    #print('Score Range',int(model.predict([temp]))-18,' to ',int(model.predict([temp]))+18)



In [ ]:
Bat_team = 'Sunrisers Hyderabad'
Bowl_team= 'Chennai Super Kings'
innings = 2
tgt = 155
overs = 6
total_score = 37
tot_wkt = 2
last_30_runs = 31
last_30_dot = 17
last_30_wkt = 1
last_30_bound = 6

wkt_prediction(df,Bat_team, Bowl_team,innings, overs, total_score, tot_wkt, last_30_runs, last_30_dot, last_30_wkt, last_30_bound,tgt)

#Csk v Srh Ipl2025
#Second Innings Wicket Prediction

6


In [ ]:
Bat_team = 'Sunrisers Hyderabad'
Bowl_team= 'Chennai Super Kings'
innings = 2
tgt = 155
overs = 6
total_score = 37
tot_wkt = 2
last_30_runs = 31
last_30_dot = 17
last_30_wkt = 1
last_30_bound = 6

score_prediction(df,Bat_team, Bowl_team,innings, overs, total_score, tot_wkt, last_30_runs, last_30_dot, last_30_wkt, last_30_bound,tgt)

#Csk v Srh Ipl2025
#Second Innings Score Prediction
#25-Apr-2025

142
Score Range 124  to  160
